## Section 1: Environment Setup

Install and import libraries for LangGraph multi-agent system.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install langchain langchain-openai langgraph langchain-core

# Import required libraries
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor, tool
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Dict, Any
import operator
from datetime import datetime, timedelta
import os
import time
import json

# TODO: Set your Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4"

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-02-01",
    temperature=0
)

print("✓ Environment configured successfully!")

## Section 2: Sample Customer Data

Extended mock database with more customers and transaction data.

In [ ]:
# Mock customer database
CUSTOMER_DATA = {
    "CUST001": {
        "name": "Alice Johnson",
        "email": "alice@example.com",
        "phone": "555-0101",
        "accounts": {
            "CHK001": {"type": "checking", "balance": 5000.00},
            "SAV001": {"type": "savings", "balance": 15000.00}
        },
        "transactions": [
            {"date": "2025-12-15", "type": "deposit", "account": "CHK001", "amount": 2000.00, "description": "Salary"},
            {"date": "2025-12-14", "type": "withdrawal", "account": "CHK001", "amount": 150.00, "description": "ATM Withdrawal"},
            {"date": "2025-12-13", "type": "purchase", "account": "CHK001", "amount": 85.50, "description": "Grocery Store"},
            {"date": "2025-12-12", "type": "transfer", "account": "SAV001", "amount": 500.00, "description": "Transfer from CHK001"},
            {"date": "2025-12-10", "type": "interest", "account": "SAV001", "amount": 25.00, "description": "Monthly Interest"},
            {"date": "2025-12-09", "type": "purchase", "account": "CHK001", "amount": 45.20, "description": "Gas Station"},
            {"date": "2025-12-08", "type": "purchase", "account": "CHK001", "amount": 120.00, "description": "Restaurant"},
            {"date": "2025-12-07", "type": "withdrawal", "account": "CHK001", "amount": 200.00, "description": "ATM Withdrawal"},
        ]
    },
    "CUST002": {
        "name": "Bob Smith",
        "email": "bob@example.com",
        "phone": "555-0102",
        "accounts": {
            "CHK002": {"type": "checking", "balance": 3200.00},
            "SAV002": {"type": "savings", "balance": 8500.00}
        },
        "transactions": [
            {"date": "2025-12-15", "type": "deposit", "account": "CHK002", "amount": 1500.00, "description": "Paycheck"},
            {"date": "2025-12-14", "type": "purchase", "account": "CHK002", "amount": 200.00, "description": "Online Shopping"},
            {"date": "2025-12-12", "type": "purchase", "account": "CHK002", "amount": 75.00, "description": "Utilities"},
        ]
    }
}

# Support knowledge base
SUPPORT_KB = {
    "hours": "Our customer service is available Monday-Friday 9AM-5PM EST.",
    "routing_number": "Your routing number is 123456789.",
    "fees": "Checking accounts have no monthly fees. Savings accounts earn 2.5% APY.",
    "overdraft": "Overdraft protection is available for $5/month. It links your savings to checking.",
    "wire_transfer": "Domestic wire transfers cost $25. International wires cost $45.",
    "card_lost": "Report lost cards immediately by calling 1-800-BANK-HELP. We'll freeze your account and send a replacement."
}

print(f"✓ Loaded {len(CUSTOMER_DATA)} customers with mock data")
print(f"✓ Loaded {len(SUPPORT_KB)} support knowledge base entries")

## Section 3: Define Tools for Each Agent

Create specialized tools for each agent type.

In [ ]:
# ============= ACCOUNT MANAGEMENT TOOLS =============

@tool
def get_account_balance(customer_id: str, account_id: str = None) -> str:
    """Get account balance for a customer. If account_id not provided, returns all accounts."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    
    if account_id:
        if account_id not in customer["accounts"]:
            return f"Error: Account {account_id} not found."
        account = customer["accounts"][account_id]
        return f"{customer['name']}'s {account['type']} account ({account_id}): ${account['balance']:,.2f}"
    else:
        result = f"Account balances for {customer['name']}:\n"
        for acc_id, acc_data in customer["accounts"].items():
            result += f"  - {acc_data['type'].capitalize()} ({acc_id}): ${acc_data['balance']:,.2f}\n"
        return result

@tool
def get_transaction_history(customer_id: str, account_id: str = None, limit: int = 5) -> str:
    """Get recent transaction history for a customer."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    transactions = customer["transactions"]
    
    if account_id:
        transactions = [t for t in transactions if t["account"] == account_id]
    
    transactions = transactions[:limit]
    
    if not transactions:
        return f"No transactions found."
    
    result = f"Recent transactions for {customer['name']}:\n"
    for txn in transactions:
        result += f"  - {txn['date']}: {txn['type'].upper()} ${txn['amount']:,.2f} - {txn['description']} ({txn['account']})\n"
    
    return result

@tool
def transfer_money(customer_id: str, from_account: str, to_account: str, amount: float) -> str:
    """Transfer money between accounts."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    
    if from_account not in customer["accounts"]:
        return f"Error: Source account {from_account} not found."
    if to_account not in customer["accounts"]:
        return f"Error: Destination account {to_account} not found."
    
    if amount <= 0:
        return f"Error: Transfer amount must be positive."
    
    if customer["accounts"][from_account]["balance"] < amount:
        return f"Error: Insufficient funds. Available: ${customer['accounts'][from_account]['balance']:,.2f}"
    
    customer["accounts"][from_account]["balance"] -= amount
    customer["accounts"][to_account]["balance"] += amount
    
    today = datetime.now().strftime("%Y-%m-%d")
    customer["transactions"].insert(0, {
        "date": today,
        "type": "transfer",
        "account": from_account,
        "amount": amount,
        "description": f"Transfer to {to_account}"
    })
    
    return f"✓ Transfer successful! ${amount:,.2f} moved from {from_account} to {to_account}.\nNew balances: {from_account}: ${customer['accounts'][from_account]['balance']:,.2f}, {to_account}: ${customer['accounts'][to_account]['balance']:,.2f}"

# Group account tools
account_tools = [get_account_balance, get_transaction_history, transfer_money]

print(f"✓ Created {len(account_tools)} account management tools")

In [ ]:
# ============= SUPPORT TOOLS =============

@tool
def get_customer_info(customer_id: str) -> str:
    """Get customer contact information."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    return f"Customer: {customer['name']}\nEmail: {customer['email']}\nPhone: {customer['phone']}"

@tool
def search_support_kb(query: str) -> str:
    """Search support knowledge base for banking information. Query can be: hours, routing_number, fees, overdraft, wire_transfer, card_lost."""
    query_lower = query.lower()
    
    # Find matching KB entries
    results = []
    for key, value in SUPPORT_KB.items():
        if key in query_lower or query_lower in key:
            results.append(f"**{key.replace('_', ' ').title()}**: {value}")
    
    if not results:
        # Return all available topics
        return f"No exact match found. Available topics: {', '.join(SUPPORT_KB.keys())}\n\nPlease ask about a specific topic."
    
    return "\n\n".join(results)

@tool
def create_support_ticket(customer_id: str, issue_description: str) -> str:
    """Create a support ticket for customer issues."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    ticket_id = f"TICKET-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    customer = CUSTOMER_DATA[customer_id]
    
    return f"✓ Support ticket created:\nTicket ID: {ticket_id}\nCustomer: {customer['name']}\nIssue: {issue_description}\nStatus: OPEN\n\nA support representative will contact you within 24 hours."

# Group support tools
support_tools = [get_customer_info, search_support_kb, create_support_ticket]

print(f"✓ Created {len(support_tools)} support tools")

In [ ]:
# ============= ANALYTICS TOOLS =============

@tool
def analyze_spending(customer_id: str, days: int = 30) -> str:
    """Analyze customer spending patterns over specified days."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    transactions = customer["transactions"]
    
    # Filter by transaction type
    spending_txns = [t for t in transactions if t["type"] in ["purchase", "withdrawal"]]
    
    if not spending_txns:
        return "No spending transactions found."
    
    total_spent = sum(t["amount"] for t in spending_txns)
    avg_transaction = total_spent / len(spending_txns)
    
    # Category breakdown
    categories = {}
    for txn in spending_txns:
        txn_type = txn["type"]
        categories[txn_type] = categories.get(txn_type, 0) + txn["amount"]
    
    result = f"Spending Analysis for {customer['name']} (Last {days} days):\n"
    result += f"  Total Spent: ${total_spent:,.2f}\n"
    result += f"  Number of Transactions: {len(spending_txns)}\n"
    result += f"  Average Transaction: ${avg_transaction:,.2f}\n\n"
    result += "Breakdown by Category:\n"
    for category, amount in categories.items():
        result += f"  - {category.capitalize()}: ${amount:,.2f}\n"
    
    return result

@tool
def calculate_savings_rate(customer_id: str) -> str:
    """Calculate customer's savings rate based on deposits vs spending."""
    if customer_id not in CUSTOMER_DATA:
        return f"Error: Customer {customer_id} not found."
    
    customer = CUSTOMER_DATA[customer_id]
    transactions = customer["transactions"]
    
    deposits = sum(t["amount"] for t in transactions if t["type"] == "deposit")
    spending = sum(t["amount"] for t in transactions if t["type"] in ["purchase", "withdrawal"])
    
    if deposits == 0:
        return "No deposit transactions found to calculate savings rate."
    
    savings_rate = ((deposits - spending) / deposits) * 100
    
    result = f"Savings Rate Analysis for {customer['name']}:\n"
    result += f"  Total Deposits: ${deposits:,.2f}\n"
    result += f"  Total Spending: ${spending:,.2f}\n"
    result += f"  Net Savings: ${deposits - spending:,.2f}\n"
    result += f"  Savings Rate: {savings_rate:.1f}%\n\n"
    
    if savings_rate > 20:
        result += "✓ Excellent! You're saving over 20% of your income."
    elif savings_rate > 10:
        result += "✓ Good savings rate. Consider increasing to 20% for better financial health."
    else:
        result += "⚠ Consider reducing spending to increase your savings rate."
    
    return result

# Group analytics tools
analytics_tools = [analyze_spending, calculate_savings_rate]

print(f"✓ Created {len(analytics_tools)} analytics tools")

## Section 4: Create Specialized Agents

Build three specialized agents with their own tools and prompts.

In [ ]:
# ============= ACCOUNT MANAGEMENT AGENT =============

account_prompt = PromptTemplate.from_template(
    """You are an Account Management Specialist. You help customers with:
- Checking account balances
- Viewing transaction history
- Transferring money between accounts

Always confirm customer ID before performing operations. Be professional and accurate.

Available tools:
{tools}

Tool names: {tool_names}

Use this format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}"""
)

account_agent = create_react_agent(
    llm=llm,
    tools=account_tools,
    prompt=account_prompt
)

account_agent_executor = AgentExecutor(
    agent=account_agent,
    tools=account_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✓ Account Management Agent created")

In [ ]:
# ============= SUPPORT AGENT =============

support_prompt = PromptTemplate.from_template(
    """You are a Customer Support Specialist. You help customers with:
- General banking questions (hours, fees, routing numbers)
- Account policies and procedures
- Creating support tickets for issues
- Providing customer information

Be friendly, helpful, and empathetic. Always search the knowledge base first.

Available tools:
{tools}

Tool names: {tool_names}

Use this format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}"""
)

support_agent = create_react_agent(
    llm=llm,
    tools=support_tools,
    prompt=support_prompt
)

support_agent_executor = AgentExecutor(
    agent=support_agent,
    tools=support_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✓ Support Agent created")

In [ ]:
# ============= ANALYTICS AGENT =============

analytics_prompt = PromptTemplate.from_template(
    """You are a Financial Analytics Specialist. You help customers with:
- Analyzing spending patterns
- Calculating savings rates
- Providing financial insights and recommendations

Provide clear, actionable insights based on the data. Use visualizations when helpful.

Available tools:
{tools}

Tool names: {tool_names}

Use this format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}"""
)

analytics_agent = create_react_agent(
    llm=llm,
    tools=analytics_tools,
    prompt=analytics_prompt
)

analytics_agent_executor = AgentExecutor(
    agent=analytics_agent,
    tools=analytics_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

print("✓ Analytics Agent created")

## Section 5: Define LangGraph State

Create the state schema for our multi-agent workflow.

In [ ]:
# Define the state for our multi-agent graph
class BankingState(TypedDict):
    """State schema for the banking multi-agent system."""
    # User input and conversation
    messages: Annotated[List[Any], operator.add]  # Messages exchanged
    user_input: str  # Original user query
    
    # Routing and agent tracking
    current_agent: str  # Which agent is currently active
    task_type: str  # Type of task (account, support, analytics)
    routing_step: int  # Track number of agent handoffs
    
    # Results and metadata
    final_answer: str  # Final response to user
    agent_trace: List[Dict[str, Any]]  # Trace of agent executions
    time_taken: float  # Total execution time

print("✓ State schema defined")

## Section 6: Build Coordinator Agent

Create a coordinator that routes requests to appropriate specialist agents.

In [ ]:
def coordinator_node(state: BankingState) -> BankingState:
    """Coordinator agent that routes requests to specialist agents."""
    user_input = state["user_input"].lower()
    
    # TODO: Implement routing logic based on keywords
    # Account-related keywords
    account_keywords = ["balance", "transaction", "transfer", "account", "money", "deposit", "withdrawal"]
    
    # Support-related keywords
    support_keywords = ["help", "support", "question", "hours", "routing", "fee", "policy", "ticket", "issue", "problem"]
    
    # Analytics-related keywords
    analytics_keywords = ["analyze", "spending", "savings", "rate", "pattern", "insight", "report", "statistics"]
    
    # Determine which agent to route to
    if any(keyword in user_input for keyword in account_keywords):
        task_type = "account_management"
        agent_name = "Account Agent"
    elif any(keyword in user_input for keyword in analytics_keywords):
        task_type = "analytics"
        agent_name = "Analytics Agent"
    elif any(keyword in user_input for keyword in support_keywords):
        task_type = "support"
        agent_name = "Support Agent"
    else:
        # Default to support for general queries
        task_type = "support"
        agent_name = "Support Agent"
    
    print(f"\n🔀 COORDINATOR: Routing to {agent_name} for {task_type} task")
    
    return {
        **state,
        "current_agent": agent_name,
        "task_type": task_type,
        "routing_step": state.get("routing_step", 0) + 1,
        "messages": [HumanMessage(content=f"Coordinator routing to {agent_name}")]
    }

print("✓ Coordinator node defined")

## Section 7: Build Specialist Agent Nodes

Create nodes for each specialist agent in the LangGraph workflow.

In [ ]:
def account_agent_node(state: BankingState) -> BankingState:
    """Execute account management agent."""
    print("\n💰 ACCOUNT AGENT: Processing request...")
    start_time = time.time()
    
    # Execute the account agent
    result = account_agent_executor.invoke({"input": state["user_input"]})
    
    end_time = time.time()
    
    # Record trace
    trace_entry = {
        "agent": "Account Agent",
        "task_type": "account_management",
        "input": state["user_input"],
        "output": result["output"],
        "time_taken": end_time - start_time
    }
    
    return {
        **state,
        "final_answer": result["output"],
        "agent_trace": state.get("agent_trace", []) + [trace_entry],
        "time_taken": state.get("time_taken", 0) + (end_time - start_time),
        "messages": [AIMessage(content=result["output"])]
    }

def support_agent_node(state: BankingState) -> BankingState:
    """Execute support agent."""
    print("\n🎧 SUPPORT AGENT: Processing request...")
    start_time = time.time()
    
    result = support_agent_executor.invoke({"input": state["user_input"]})
    
    end_time = time.time()
    
    trace_entry = {
        "agent": "Support Agent",
        "task_type": "support",
        "input": state["user_input"],
        "output": result["output"],
        "time_taken": end_time - start_time
    }
    
    return {
        **state,
        "final_answer": result["output"],
        "agent_trace": state.get("agent_trace", []) + [trace_entry],
        "time_taken": state.get("time_taken", 0) + (end_time - start_time),
        "messages": [AIMessage(content=result["output"])]
    }

def analytics_agent_node(state: BankingState) -> BankingState:
    """Execute analytics agent."""
    print("\n📊 ANALYTICS AGENT: Processing request...")
    start_time = time.time()
    
    result = analytics_agent_executor.invoke({"input": state["user_input"]})
    
    end_time = time.time()
    
    trace_entry = {
        "agent": "Analytics Agent",
        "task_type": "analytics",
        "input": state["user_input"],
        "output": result["output"],
        "time_taken": end_time - start_time
    }
    
    return {
        **state,
        "final_answer": result["output"],
        "agent_trace": state.get("agent_trace", []) + [trace_entry],
        "time_taken": state.get("time_taken", 0) + (end_time - start_time),
        "messages": [AIMessage(content=result["output"])]
    }

print("✓ Specialist agent nodes defined")

## Section 8: Build the LangGraph Workflow

Connect all nodes into a cohesive multi-agent workflow.

In [ ]:
# Define routing logic
def route_to_agent(state: BankingState) -> str:
    """Determine which agent node to execute next."""
    task_type = state.get("task_type", "support")
    
    if task_type == "account_management":
        return "account_agent"
    elif task_type == "analytics":
        return "analytics_agent"
    else:
        return "support_agent"

# Build the graph
workflow = StateGraph(BankingState)

# Add nodes
workflow.add_node("coordinator", coordinator_node)
workflow.add_node("account_agent", account_agent_node)
workflow.add_node("support_agent", support_agent_node)
workflow.add_node("analytics_agent", analytics_agent_node)

# Set entry point
workflow.set_entry_point("coordinator")

# Add conditional routing from coordinator to specialist agents
workflow.add_conditional_edges(
    "coordinator",
    route_to_agent,
    {
        "account_agent": "account_agent",
        "analytics_agent": "analytics_agent",
        "support_agent": "support_agent"
    }
)

# All specialist agents end after execution
workflow.add_edge("account_agent", END)
workflow.add_edge("support_agent", END)
workflow.add_edge("analytics_agent", END)

# Compile the graph
app = workflow.compile()

print("✓ Multi-agent workflow compiled successfully!")

## Section 9: Test Multi-Agent System

Run various test scenarios through the multi-agent workflow.

In [ ]:
def run_multi_agent_query(query: str, customer_id: str = "CUST001"):
    """Execute a query through the multi-agent system."""
    print("=" * 80)
    print(f"USER QUERY: {query}")
    print("=" * 80)
    
    # Prepare initial state
    initial_state = {
        "messages": [],
        "user_input": query,
        "current_agent": "Coordinator",
        "task_type": "unknown",
        "routing_step": 0,
        "final_answer": "",
        "agent_trace": [],
        "time_taken": 0.0
    }
    
    # Execute workflow
    result = app.invoke(initial_state)
    
    # Display results
    print("\n" + "=" * 80)
    print("EXECUTION TRACE:")
    print("=" * 80)
    for i, trace in enumerate(result["agent_trace"], 1):
        print(f"\nStep {i}: {trace['agent']}")
        print(f"  Task Type: {trace['task_type']}")
        print(f"  Time Taken: {trace['time_taken']:.2f}s")
    
    print("\n" + "=" * 80)
    print("FINAL ANSWER:")
    print("=" * 80)
    print(result["final_answer"])
    
    print(f"\n⏱ Total Time: {result['time_taken']:.2f}s")
    print(f"🔀 Routing Steps: {result['routing_step']}")
    
    return result

print("✓ Test harness ready")

In [ ]:
# Test 1: Account balance query (should route to Account Agent)
result1 = run_multi_agent_query(
    "What is my account balance? I'm customer CUST001."
)

In [ ]:
# Test 2: Support query (should route to Support Agent)
result2 = run_multi_agent_query(
    "What are your business hours?"
)

In [ ]:
# Test 3: Analytics query (should route to Analytics Agent)
result3 = run_multi_agent_query(
    "Analyze my spending patterns for customer CUST001"
)

In [ ]:
# Test 4: Complex transfer (should route to Account Agent)
result4 = run_multi_agent_query(
    "I'm customer CUST001. Transfer $1000 from my checking account CHK001 to savings SAV001."
)

In [ ]:
# Test 5: Savings rate calculation (should route to Analytics Agent)
result5 = run_multi_agent_query(
    "Calculate my savings rate for customer CUST001"
)

## Section 10: Visualize Agent Flow

Analyze and visualize the multi-agent execution patterns.

In [ ]:
def analyze_agent_performance(results_list: List[Dict]) -> None:
    """Analyze performance across multiple agent executions."""
    print("\n" + "=" * 80)
    print("MULTI-AGENT SYSTEM PERFORMANCE ANALYSIS")
    print("=" * 80)
    
    # Aggregate stats by agent
    agent_stats = {}
    
    for result in results_list:
        for trace in result["agent_trace"]:
            agent_name = trace["agent"]
            if agent_name not in agent_stats:
                agent_stats[agent_name] = {
                    "count": 0,
                    "total_time": 0,
                    "avg_time": 0
                }
            
            agent_stats[agent_name]["count"] += 1
            agent_stats[agent_name]["total_time"] += trace["time_taken"]
    
    # Calculate averages
    for agent_name, stats in agent_stats.items():
        stats["avg_time"] = stats["total_time"] / stats["count"]
    
    # Display stats
    print("\nAgent Usage Statistics:")
    print("-" * 80)
    print(f"{'Agent':<25} {'Invocations':<15} {'Avg Time (s)':<15} {'Total Time (s)':<15}")
    print("-" * 80)
    
    for agent_name, stats in sorted(agent_stats.items()):
        print(f"{agent_name:<25} {stats['count']:<15} {stats['avg_time']:<15.2f} {stats['total_time']:<15.2f}")
    
    # Overall stats
    total_queries = len(results_list)
    total_time = sum(r["time_taken"] for r in results_list)
    avg_time = total_time / total_queries if total_queries > 0 else 0
    
    print("\n" + "=" * 80)
    print("Overall System Statistics:")
    print("-" * 80)
    print(f"Total Queries Processed: {total_queries}")
    print(f"Total Execution Time: {total_time:.2f}s")
    print(f"Average Query Time: {avg_time:.2f}s")
    print("=" * 80)

# Run analysis on all test results (uncomment after running all tests)
# analyze_agent_performance([result1, result2, result3, result4, result5])

print("✓ Analysis tools ready")

## Section 11: Advanced - Add Agent Collaboration

Extend the system to allow agents to collaborate on complex queries.

In [ ]:
# TODO: Advanced exercise - Implement agent collaboration
# Example: A query like "Analyze my spending and suggest how much to transfer to savings"
# would require both Analytics Agent (to analyze) and Account Agent (to transfer)

def collaborative_coordinator_node(state: BankingState) -> BankingState:
    """Enhanced coordinator that can route to multiple agents sequentially."""
    # This is a placeholder for an advanced implementation
    # where the coordinator can chain multiple agents together
    pass

print("💡 Exercise: Implement agent collaboration for complex multi-step queries!")

## Workshop Complete! 🎉

You've successfully built a multi-agent banking system with LangGraph that:
- ✓ Routes requests to specialized agents (Account, Support, Analytics)
- ✓ Manages state across the workflow
- ✓ Tracks execution and performance metrics
- ✓ Handles different types of banking queries intelligently

### Key Concepts Learned:
1. **Agent Specialization**: Each agent focuses on specific tasks with dedicated tools
2. **LangGraph Workflow**: Using StateGraph for multi-agent orchestration
3. **Conditional Routing**: Coordinator intelligently routes based on query analysis
4. **State Management**: Tracking conversation state, traces, and metadata
5. **Performance Monitoring**: Measuring execution time and agent usage

### Next Steps:
1. **Add More Agents**: Create loan agent, investment advisor, fraud detection agent
2. **Implement Collaboration**: Allow agents to work together on complex queries
3. **Add Memory**: Implement conversation memory across sessions
4. **Enhance Routing**: Use LLM-based routing instead of keyword matching
5. **Database Integration**: Connect to real SQL database
6. **Add Logging**: Implement comprehensive logging to database (like in your production system)
7. **Web Interface**: Build Flask API and frontend for customer interaction
8. **Security**: Add authentication, authorization, and audit logging

### Comparison to Single Agent:
- **Single Agent**: All tools in one agent, simpler but less organized
- **Multi-Agent**: Specialized agents, better scalability and maintainability
- **When to use**: Multi-agent for complex domains with distinct responsibilities

Great job! You now understand the fundamentals of building production-grade multi-agent systems! 🚀